In [19]:
import numpy as np
import random
from tqdm.notebook import tqdm

In [20]:
%run tetris-field.ipynb
%run tetris-field-rendered.ipynb
%run tetris-bag.ipynb

          
          
          
          
          
          
          
          
          
          
          
          
          
          
█████     
████      
████ █████
████  ████
████ █████
██████████


In [21]:
SCORING = {
    'NORMAL': {
        0: 0,
        1: 50,
        2: 150,
        3: 300,
        4: 800
    },
    
    'TSPIN': {
        0: 50,
        1: 600,
        2: 1200,
        3: 1600,
        4: 2600
    },
    
    'BACKTOBACK_MULTIPLIER': 1.5,
    'COMBO': 50,
    'ALL_CLEAR': 3500,
    
    'GAME_OVER': -5000
}

In [22]:
class TetrisEnv():
    def __init__(self):
        self.field = Field()
        self.bag = Bag()
        
        self.current_piece = self.bag.next_piece()
        
        self.current_combo = 1
        self.current_btb = 1
        
        # game statistics
        self.score = 0
        self.moves = 0
        self.clears = [0, 0, 0, 0]
        self.tspins = 0
        self.all_clears = 0
        
        
    def get_next_states(self):
        '''Returns all possible follow states by placing the current piece at (nearly) all possible positions.'''
        next_states, next_clears = self.field.get_follow_states(Piece(self.current_piece))
        # current_states = [self.field.get_current_state() for _ in range(len(next_states))]
        scores = [SCORING['GAME_OVER'] if clear is None else self._calculate_score(*clear, self.current_combo, self.current_btb) for clear in next_clears]
        dones = [clear is None for clear in next_clears]
        queue = []
        
        return (next_states, scores, next_clears, dones)
    
    def get_current_state(self):
        return self.field.get_current_state()
    
    def get_next_queue(self, length=5):
        return self.bag.peek_pieces(length)
    
    
    def step(self, next_state, next_clear, next_score):
        self.field.field_data = next_state
        self.current_piece = self.bag.next_piece()
        
        cleared_rows = next_clear[0]
        is_tspin = next_clear[1]
        is_all_clear = next_clear[2]
        
            
        # update back-to-back
        if is_tspin or cleared_rows == 4:
            self.current_btb += 1
        elif cleared_rows > 0:
            self.current_btb = 1
        
        # update combo
        if cleared_rows > 0:
            self.current_combo += 1
            self.clears[cleared_rows - 1] += 1
        else:
            self.current_combo = 1
            
        # update this run's score
        self.score += next_score
        
        if is_tspin:
            self.tspins += 1
        if is_all_clear:
            self.all_clears += 1
            
        self.moves += 1
        
        # print('==== DOING STEP =====')
        # self.field._print_board(next_state)
        # print(next_clear)
        # print()
    
    def _calculate_score(self, cleared_lines, t_spin, all_clear, combo, btb):
        """Calculates the score by considering the number of cleared lines, whether a t-spin was performed and the current combo and back-to-back streaks."""
        # get base score
        score = SCORING['TSPIN' if t_spin else 'NORMAL'][cleared_lines]

        if cleared_lines > 0 and btb > 1:
            score *= SCORING['BACKTOBACK_MULTIPLIER'] * (btb / 2)

        if combo > 1:
            score += SCORING['COMBO'] * (combo - 1)

        if all_clear:
            score += SCORING['ALL_CLEAR']

        return score
    
    def _print_state(self, s):
        self.field._print_board(s)
    def _print_current_state(self):
        self._print_state(self.get_current_state())
    
    
    def observation(self):
        pass
        
    def reset(self):
        self.field.reset()
        self.bag.reset()
        
        self.current_piece = self.bag.next_piece()
        
        self.current_combo = 1
        self.current_btb = 1
        
        self.score = 0
        self.moves = 0
        self.clears = [0, 0, 0, 0]
        self.tspins = 0
        self.all_clears = 0
    
    def close(self):
        pass
        
    def render(self, mode=None):
        pass
        

In [23]:
class TetrisEnvRendered():
    def __init__(self):
        self.field = FieldRendered()
        self.bag = Bag()
        
        self.current_piece = self.bag.next_piece()
        
        self.current_combo = 1
        self.current_btb = 1
        
        # game statistics
        self.score = 0
        self.moves = 0
        self.clears = [0, 0, 0, 0]
        self.tspins = 0
        self.all_clears = 0
        
        
    def get_next_states(self):
        '''Returns all possible follow states by placing the current piece at (nearly) all possible positions.'''
        next_states, next_states_pretty, next_clears, next_moves = self.field.get_follow_states(Piece(self.current_piece))

        # current_states = [self.field.get_current_state() for _ in range(len(next_states))]
        scores = [SCORING['GAME_OVER'] if clear is None else self._calculate_score(*clear, self.current_combo, self.current_btb) for clear in next_clears]
        dones = [clear is None for clear in next_clears]
        queue = []
        
        return (next_states, next_states_pretty, scores, next_clears, dones, next_moves)
    
    def get_current_state(self):
        return self.field.get_current_state()
    
    def get_current_state_pretty(self):
        return self.field.get_current_state_pretty()
    
    def get_next_queue(self, length=5):
        return self.bag.peek_pieces(length)
    
    
    def step(self, next_state, next_state_pretty, next_clear, next_score):
        self.field.field_data = next_state
        self.field.field_data_pretty = next_state_pretty
        
        self.current_piece = self.bag.next_piece()
        
        cleared_rows = next_clear[0]
        is_tspin = next_clear[1]
        is_all_clear = next_clear[2]
        
            
        # update back-to-back
        if is_tspin or cleared_rows == 4:
            self.current_btb += 1
        elif cleared_rows > 0:
            self.current_btb = 1
        
        # update combo
        if cleared_rows > 0:
            self.current_combo += 1
            self.clears[cleared_rows - 1] += 1
        else:
            self.current_combo = 1
            
        # update this run's score
        self.score += next_score
        
        if is_tspin:
            self.tspins += 1
        if is_all_clear:
            self.all_clears += 1
            
        self.moves += 1
        
        # print('==== DOING STEP =====')
        # self.field._print_board(next_state)
        # print(next_clear)
        # print()
    
    def _calculate_score(self, cleared_lines, t_spin, all_clear, combo, btb):
        """Calculates the score by considering the number of cleared lines, whether a t-spin was performed and the current combo and back-to-back streaks."""
        # get base score
        score = SCORING['TSPIN' if t_spin else 'NORMAL'][cleared_lines]

        if cleared_lines > 0 and btb > 1:
            score *= SCORING['BACKTOBACK_MULTIPLIER'] * (btb / 2)

        if combo > 1:
            score += SCORING['COMBO'] * (combo - 1)

        if all_clear:
            score += SCORING['ALL_CLEAR']

        return score
    
    def _print_state(self, s):
        self.field._print_board(s)
    def _print_current_state(self):
        self._print_state(self.get_current_state())
    
    
    def observation(self):
        pass
        
    def reset(self):
        self.field.reset()
        self.bag.reset()
        
        self.current_piece = self.bag.next_piece()
        
        self.current_combo = 1
        self.current_btb = 1
        
        self.score = 0
        self.moves = 0
        self.clears = [0, 0, 0, 0]
        self.tspins = 0
        self.all_clears = 0
    
    def close(self):
        pass
        
    def render(self, mode=None):
        pass
        

In [24]:
# env = TetrisEnvRendered()

# env.reset()
# states_to_render = []

# while True:
#     states, states_pretty, scores, clears, dones, moves = env.get_next_states()
    
#     chosen_index = random.choice(range(len(states))) # get_best_state(torch.from_numpy(states.reshape(-1, 1, 20, 10)).float().to(device), False)

#     states_to_render.append((states_pretty[chosen_index], moves[chosen_index], env.current_piece))

#     if dones[chosen_index]:
#         print(f'Score: {env.score}')
#         print(f'Clears: {env.clears}, t-spins: {env.tspins}, alll_clears: {env.all_clears}')
#         break
#     else:
#         env.step(states[chosen_index], states_pretty[chosen_index], clears[chosen_index], scores[chosen_index])    

Score: 0
Clears: [0, 0, 0, 0], t-spins: 0, alll_clears: 0


In [25]:
# import time
# gui.gui.start_gui()
# g = gui.gui.Gui()

# last_state = np.ones((20,10)) * -1

# for state, move, piece in states_to_render:
#     p = Piece(piece)
    
#     for m in move:
#         if m == 'ml':
#             p.pos[1] -= 1
#         if m == 'mr':
#             p.pos[1] += 1
#         if m == 'sd':
#             p.pos[0] += 1

#         if m == 'rr':
#             p.rot = (p.rot + 1) % p.pdata.num_rot
#         if m == 'rl':
#             p.rot = (p.rot - 1) % p.pdata.num_rot
                      
#         g.draw(last_state, p)
    
#     last_state = state

In [26]:
# env = TetrisEnv()

# print('Current piece:', env.current_piece)
# env._print_current_state()

# scored_points = False
# counter = 0
# scs = []

# with tqdm(total=100000) as pbar:
#     while not scored_points:
#         for i in range(40):
#             states, scores, clears, dones = env.get_next_states()

#             chosen_index = random.randint(0, len(states) - 1)

#             if dones[chosen_index]:
#                 if counter % 5 == 1:
#                     print(f'Score: {env.score}, #: {len(scs)}, min: {np.min(scs)}, max: {np.max(scs)}, avg: {np.average(scs)}')
                
#                 # clear_output(wait=True)
#                 # print(env.score)
#                 scs.append(env.score)
#                 env.reset()
#                 break
#             else:
#                 env.step(states[chosen_index], clears[chosen_index], scores[chosen_index])
            
#             if clears[chosen_index][0] > 1:
#                 scored_points = True
#                 print('YOOOOO', clears[chosen_index][0])
#                 break

           
#         counter += 1
#         pbar.update(1)

    
# print('took ', counter)